# Extract features, retrain Sherlock and generate predictions.

### Necessary imports

In [42]:
import pandas as pd
import numpy as np
import sys
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
sys.path.append("..")

In [7]:
from src.features.build_features import build_features
from src.deploy.train_sherlock import train_sherlock
from src.deploy.predict_sherlock import predict_sherlock

### Load small raw data sample and corresponding labels

In [3]:
data = pd.read_csv('../data/raw/test_values.csv', sep=',', index_col=0, header=None)
labs = pd.read_csv('../data/raw/test_labels.csv', sep=',', index_col=0, header=None)

In [4]:
data.head()

,1
0,
20368,"['Central Missouri', 'unattached', 'unattached..."
664102,"[95, 100, 95, 89, 84, 91, 88, 94, 75, 78, 90, ..."
366813,"['Katie Crews', 'Christian Hiraldo', 'Alex Est..."
530567,"['Christian', 'Non-Christian', 'Unreported', '..."
176253,"['AAF-McQuay Canada Inc.', 'AAF-McQuay Canada ..."


In [5]:
labs.head()

,1
0,
20368,affiliation
664102,weight
366813,jockey
530567,religion
176253,company


### Extract features of raw data samples from dataframes and save preprocessed data

In [ ]:
# Load pre-extracted features of sample file
with open('../data/processed/X_train.data', 'rb') as f:
    X_train = pickle.load(f)
    
with open('../data/processed/y_train.data', 'rb') as f:
    y_train = pickle.load(f)

In [ ]:
# Extract features of sample file

In [36]:
X_train = build_features(data)
y_train = labs.values.flatten()
print('Extracted features.')

### Train sherlock on new data

In [12]:
# For simplicity provide X_train as validation set.
train_sherlock(X_train, y_train, X_train, y_train, 'retrain_minimal_sample') 
print('Trained new model.')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Successfully loaded and compiled model, now fitting.
Train on 1001 samples, validate on 1001 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1001/1001 [==============================] - 2s 2ms/sample - loss: 4.8763 - categorical_accuracy: 0.0180 - val_loss: 4.8541 - val_categorical_accuracy: 0.0180
Epoch 2/100
1001/1001 [==============================] - 1s 514us/sample - loss: 4.7499 - categorical_accuracy: 0.0200 - val_loss: 4.7045 - val_categorical_accuracy: 0.0140
Epoch 3/100
1001/1001 [==============================] - 1s 515us/sample - loss: 4.6846 - categorical_accuracy: 0.0200 - val_loss: 4.6379 - val_categorical_accuracy: 0.0210
Epoch 4/100
1001/1001 [==============================] - 1s 503us/sample - loss: 4.5849 - categorical_accuracy: 0.0370 - val_loss: 4.5974 - val_cat

Epoch 42/100
1001/1001 [==============================] - 1s 501us/sample - loss: 2.7735 - categorical_accuracy: 0.4466 - val_loss: 3.4966 - val_categorical_accuracy: 0.4036
Epoch 43/100
1001/1001 [==============================] - 0s 498us/sample - loss: 2.7328 - categorical_accuracy: 0.4705 - val_loss: 3.4496 - val_categorical_accuracy: 0.4126
Epoch 44/100
1001/1001 [==============================] - 1s 521us/sample - loss: 2.6510 - categorical_accuracy: 0.4915 - val_loss: 3.4017 - val_categorical_accuracy: 0.4166
Epoch 45/100
1001/1001 [==============================] - 1s 509us/sample - loss: 2.6377 - categorical_accuracy: 0.4795 - val_loss: 3.3529 - val_categorical_accuracy: 0.4246
Epoch 46/100
1001/1001 [==============================] - 1s 500us/sample - loss: 2.5233 - categorical_accuracy: 0.5175 - val_loss: 3.3034 - val_categorical_accuracy: 0.4376
Epoch 47/100
1001/1001 [==============================] - 1s 520us/sample - loss: 2.5030 - categorical_accuracy: 0.5245 - val_loss

1001/1001 [==============================] - 1s 521us/sample - loss: 0.9175 - categorical_accuracy: 0.9021 - val_loss: 1.4591 - val_categorical_accuracy: 0.7762
Epoch 89/100
1001/1001 [==============================] - 1s 501us/sample - loss: 0.8552 - categorical_accuracy: 0.9211 - val_loss: 1.4308 - val_categorical_accuracy: 0.7822
Epoch 90/100
1001/1001 [==============================] - 0s 480us/sample - loss: 0.8417 - categorical_accuracy: 0.9161 - val_loss: 1.4034 - val_categorical_accuracy: 0.7852
Epoch 91/100
1001/1001 [==============================] - 1s 514us/sample - loss: 0.8375 - categorical_accuracy: 0.9241 - val_loss: 1.3792 - val_categorical_accuracy: 0.7852
Epoch 92/100
1001/1001 [==============================] - 0s 486us/sample - loss: 0.8008 - categorical_accuracy: 0.9251 - val_loss: 1.3567 - val_categorical_accuracy: 0.7902
Epoch 93/100
1001/1001 [==============================] - 0s 497us/sample - loss: 0.7783 - categorical_accuracy: 0.9431 - val_loss: 1.3330 - va

### Generate predictions with the retrained model

In [47]:
# Predict labels using the retrained model (with nn_id retrain_minimal_sample)
predicted_labels = predict_sherlock(X_train, 'retrain_minimal_sample')
print('Predicted labels: ', predicted_labels, 'true labels: ', y_train)

Predicted labels:  ['affiliation' 'weight' 'jockey' ... 'company' 'plays' 'team Name'] true labels:  ['affiliation' 'weight' 'jockey' ... 'company' 'plays' 'continent']


In [50]:
f1_score(y_train, predicted_labels, average='weighted')

C:\Users\hulse504\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.8311504866717553